In [ ]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import matplotlib.pyplot as plt
np.bool8 = np.bool_
# 策略网络
class PolicyNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(PolicyNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(state_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, action_size),
            nn.Softmax(dim=-1)
        )
        
        # 使用正确的初始化方法
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.constant_(module.bias, 0.0)
    
    def forward(self, x):
        return self.network(x)

class PolicyGradientAgent:
    def __init__(self, state_size, action_size):
        self.policy = PolicyNetwork(state_size, action_size)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=0.005)
        self.gamma = 0.99
        
    def select_action(self, state):
        state = torch.FloatTensor(state)
        # 添加探索机制
        if np.random.random() < 0.5:
            action = np.random.randint(0, 4)
            probs = self.policy(state)
            return action, torch.log(probs[action]+1e-8)
            
        
        probs = self.policy(state)
        distribution = Categorical(probs)
        action = distribution.sample()
        
        # 保存log概率用于后续训练
        return action.item(), distribution.log_prob(action)
    
    def update(self, rewards, log_probs):
        # 计算折扣回报
        returns = []
        R = 0
        for r in reversed(rewards):
            R = r + self.gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns)
        
        # 标准化回报以减少方差
        returns = (returns - returns.mean()) / (returns.std() + 1e-8)
        
        # 计算策略损失
        policy_loss = []
        for log_prob, R in zip(log_probs, returns):
            policy_loss.append(-log_prob * R)
        policy_loss = torch.stack(policy_loss).sum()
        
        # 优化策略网络
        self.optimizer.zero_grad()
        policy_loss.backward()
        self.optimizer.step()
        return policy_loss

def train(env, agent, num_episodes):
    episode_rewards = []
    max_steps_per_episode = 1000  # 增加步数限制
    best_reward = float('-inf')
    
    for episode in range(num_episodes):
        state_tuple = env.reset()
        state = state_tuple[0] if isinstance(state_tuple, tuple) else state_tuple
        
        rewards = []
        log_probs = []
        episode_reward = 0
        done = False
        steps = 0
        
        while not done and steps < max_steps_per_episode:
            steps += 1
            state_onehot = np.zeros(48)
            state_onehot[state] = 1
            
            action, log_prob = agent.select_action(state_onehot)
            
            step_result = env.step(action)
            if len(step_result) == 4:
                next_state, reward, done, _ = step_result
            else:
                next_state, reward, terminated, truncated, _ = step_result
                done = terminated or truncated
            
            # 修改奖励结构
            if reward == -100:  # 掉入悬崖
                reward = -10  # 减小惩罚力度，避免过度规避风险
            elif done and next_state == 47:  # 到达目标状态（右下角）
                reward = 10  # 给予明显的正向奖励
            
            rewards.append(reward)
            log_probs.append(log_prob)
            episode_reward += reward
            
            state = next_state
        
        avg_loss = agent.update(rewards, log_probs)
        episode_rewards.append(episode_reward)
        
        # 更新最佳奖励
        if episode_reward > best_reward:
            best_reward = episode_reward
        
        if (episode + 1) % 10 == 0:
            avg_reward = np.mean(episode_rewards[-100:])
            success_rate = len([r for r in episode_rewards[-100:] if r > -100]) / 100
            print(f"Episode {episode + 1}")
            print(f"Steps: {steps}")
            print(f"Average Reward: {avg_reward:.2f}")
            print(f"Success Rate: {success_rate:.2%}")
            print(f"Best Reward: {best_reward:.2f}")
            print(f'length: {len(log_probs)}')
            print(f'loss: {avg_loss}')
            print("-" * 50)
    
    return episode_rewards

def plot_results(rewards):
    plt.figure(figsize=(12, 6))
    plt.plot(rewards, alpha=0.6, label='Episode Reward')
    
    # 添加移动平均线
    window_size = 100
    moving_avg = np.convolve(rewards, np.ones(window_size)/window_size, mode='valid')
    plt.plot(range(window_size-1, len(rewards)), moving_avg, label='Moving Average')
    
    plt.title('Training Progress')
    plt.xlabel('Episode')
    plt.ylabel('Episode Reward')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

def evaluate_agent(env, agent, num_episodes=10):
    """评估智能体的性能"""
    total_rewards = []
    total_steps = []
    success_count = 0
    
    for i in range(num_episodes):
        state_tuple = env.reset()
        state = state_tuple[0] if isinstance(state_tuple, tuple) else state_tuple
        episode_reward = 0
        steps = 0
        done = False
        
        while not done:
            state_onehot = np.zeros(48)
            state_onehot[state] = 1
            
            # 评估时使用最可能的动作
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state_onehot)
                probs = agent.policy(state_tensor)
                action = probs.argmax().item()
            
            step_result = env.step(action)
            if len(step_result) == 4:
                next_state, reward, done, _ = step_result
            else:
                next_state, reward, terminated, truncated, _ = step_result
                done = terminated or truncated
            
            episode_reward += reward
            steps += 1
            state = next_state
            
            if done and reward != -100:  # 成功到达目标
                success_count += 1
        
        total_rewards.append(episode_reward)
        total_steps.append(steps)
        print(f"Evaluation Episode {i+1}: Reward = {episode_reward}, Steps = {steps}")
    
    avg_reward = np.mean(total_rewards)
    avg_steps = np.mean(total_steps)
    success_rate = success_count / num_episodes
    
    print("\nEvaluation Results:")
    print(f"Average Reward: {avg_reward:.2f}")
    print(f"Average Steps: {avg_steps:.2f}")
    print(f"Success Rate: {success_rate:.2%}")
    return avg_reward

if __name__ == "__main__":
    # 创建环境
    env = gym.make('CliffWalking-v0')
    
    # 初始化智能体
    state_size = 48  # CliffWalking-v0的状态空间大小
    action_size = 4  # 上下左右四个动作
#     agent = PolicyGradientAgent(state_size, action_size)
    
    # 训练
    num_episodes = 20000
    rewards = train(env, agent, num_episodes)
    
    # 绘制训练结果
    plot_results(rewards)
    
    # 评估智能体
    print("\nEvaluating trained agent...")
    evaluate_agent(env, agent, num_episodes=10)
    
    # 关闭环境
    env.close()

Episode 10
Steps: 102
Average Reward: -96.00
Success Rate: 8.00%
Best Reward: -24.00
length: 102
loss: -13.673029899597168
--------------------------------------------------
Episode 20
Steps: 52
Average Reward: -77.60
Success Rate: 17.00%
Best Reward: -24.00
length: 52
loss: -5.387212753295898
--------------------------------------------------
Episode 30
Steps: 42
Average Reward: -78.10
Success Rate: 25.00%
Best Reward: -24.00
length: 42
loss: -40.857627868652344
--------------------------------------------------
Episode 40
Steps: 49
Average Reward: -73.38
Success Rate: 34.00%
Best Reward: -13.00
length: 49
loss: -62.922325134277344
--------------------------------------------------
Episode 50
Steps: 80
Average Reward: -74.20
Success Rate: 42.00%
Best Reward: -13.00
length: 80
loss: -112.71363830566406
--------------------------------------------------
Episode 60
Steps: 41
Average Reward: -75.05
Success Rate: 50.00%
Best Reward: -13.00
length: 41
loss: -55.15721130371094
--------------

Episode 500
Steps: 68
Average Reward: -73.09
Success Rate: 74.00%
Best Reward: -4.00
length: 68
loss: -2.489297866821289
--------------------------------------------------
Episode 510
Steps: 77
Average Reward: -70.10
Success Rate: 77.00%
Best Reward: -4.00
length: 77
loss: -60.235836029052734
--------------------------------------------------
Episode 520
Steps: 32
Average Reward: -68.75
Success Rate: 79.00%
Best Reward: -4.00
length: 32
loss: -35.79461669921875
--------------------------------------------------
Episode 530
Steps: 209
Average Reward: -73.93
Success Rate: 76.00%
Best Reward: -4.00
length: 209
loss: 1.5875740051269531
--------------------------------------------------
Episode 540
Steps: 70
Average Reward: -79.19
Success Rate: 72.00%
Best Reward: -4.00
length: 70
loss: -28.375736236572266
--------------------------------------------------
Episode 550
Steps: 76
Average Reward: -79.36
Success Rate: 72.00%
Best Reward: -4.00
length: 76
loss: -43.46131134033203
---------------

Episode 990
Steps: 53
Average Reward: -76.32
Success Rate: 73.00%
Best Reward: -4.00
length: 53
loss: -51.26332092285156
--------------------------------------------------
Episode 1000
Steps: 174
Average Reward: -84.32
Success Rate: 68.00%
Best Reward: -4.00
length: 174
loss: 59.04505920410156
--------------------------------------------------
Episode 1010
Steps: 77
Average Reward: -81.28
Success Rate: 70.00%
Best Reward: -4.00
length: 77
loss: 7.991682052612305
--------------------------------------------------
Episode 1020
Steps: 132
Average Reward: -88.08
Success Rate: 66.00%
Best Reward: -4.00
length: 132
loss: 0.7059941291809082
--------------------------------------------------
Episode 1030
Steps: 72
Average Reward: -89.18
Success Rate: 65.00%
Best Reward: -4.00
length: 72
loss: -56.774452209472656
--------------------------------------------------
Episode 1040
Steps: 89
Average Reward: -90.68
Success Rate: 67.00%
Best Reward: -4.00
length: 89
loss: 53.318546295166016
-----------

Episode 1480
Steps: 105
Average Reward: -87.37
Success Rate: 63.00%
Best Reward: -4.00
length: 105
loss: -10.206717491149902
--------------------------------------------------
Episode 1490
Steps: 150
Average Reward: -87.38
Success Rate: 64.00%
Best Reward: -4.00
length: 150
loss: -247.12835693359375
--------------------------------------------------
Episode 1500
Steps: 109
Average Reward: -90.60
Success Rate: 64.00%
Best Reward: -4.00
length: 109
loss: -23.224885940551758
--------------------------------------------------
Episode 1510
Steps: 35
Average Reward: -88.63
Success Rate: 66.00%
Best Reward: -4.00
length: 35
loss: -13.715934753417969
--------------------------------------------------
Episode 1520
Steps: 361
Average Reward: -94.48
Success Rate: 64.00%
Best Reward: -4.00
length: 361
loss: -110.84127044677734
--------------------------------------------------
Episode 1530
Steps: 72
Average Reward: -94.23
Success Rate: 64.00%
Best Reward: -4.00
length: 72
loss: -53.06963348388672


Episode 1950
Steps: 61
Average Reward: -86.36
Success Rate: 71.00%
Best Reward: -4.00
length: 61
loss: 24.753934860229492
--------------------------------------------------
Episode 1960
Steps: 88
Average Reward: -85.86
Success Rate: 70.00%
Best Reward: -4.00
length: 88
loss: -102.64473724365234
--------------------------------------------------
Episode 1970
Steps: 72
Average Reward: -84.74
Success Rate: 72.00%
Best Reward: -4.00
length: 72
loss: -42.15336608886719
--------------------------------------------------
Episode 1980
Steps: 107
Average Reward: -81.96
Success Rate: 74.00%
Best Reward: -4.00
length: 107
loss: -14.351997375488281
--------------------------------------------------
Episode 1990
Steps: 38
Average Reward: -81.68
Success Rate: 73.00%
Best Reward: -4.00
length: 38
loss: -17.274147033691406
--------------------------------------------------
Episode 2000
Steps: 51
Average Reward: -81.62
Success Rate: 74.00%
Best Reward: -4.00
length: 51
loss: -75.03336334228516
--------

...

Episode 14490
Steps: 60
Average Reward: -90.45
Success Rate: 68.00%
Best Reward: -4.00
length: 60
loss: -32.10600280761719
--------------------------------------------------
Episode 14500
Steps: 114
Average Reward: -88.68
Success Rate: 67.00%
Best Reward: -4.00
length: 114
loss: -11.369811058044434
--------------------------------------------------
Episode 14510
Steps: 25
Average Reward: -84.12
Success Rate: 70.00%
Best Reward: -4.00
length: 25
loss: -15.798325538635254
--------------------------------------------------
Episode 14520
Steps: 153
Average Reward: -87.61
Success Rate: 70.00%
Best Reward: -4.00
length: 153
loss: -13.35708999633789
--------------------------------------------------
Episode 14530
Steps: 51
Average Reward: -82.27
Success Rate: 71.00%
Best Reward: -4.00
length: 51
loss: -57.2978515625
--------------------------------------------------
Episode 14540
Steps: 92
Average Reward: -85.84
Success Rate: 68.00%
Best Reward: -4.00
length: 92
loss: -76.85014343261719
-----

Episode 14960
Steps: 104
Average Reward: -102.57
Success Rate: 60.00%
Best Reward: -4.00
length: 104
loss: 19.812196731567383
--------------------------------------------------
Episode 14970
Steps: 45
Average Reward: -101.11
Success Rate: 60.00%
Best Reward: -4.00
length: 45
loss: -27.111740112304688
--------------------------------------------------
Episode 14980
Steps: 53
Average Reward: -103.70
Success Rate: 59.00%
Best Reward: -4.00
length: 53
loss: -105.58526611328125
--------------------------------------------------
Episode 14990
Steps: 196
Average Reward: -105.51
Success Rate: 59.00%
Best Reward: -4.00
length: 196
loss: -62.96922302246094
--------------------------------------------------
Episode 15000
Steps: 29
Average Reward: -105.69
Success Rate: 59.00%
Best Reward: -4.00
length: 29
loss: -55.42259979248047
--------------------------------------------------
Episode 15010
Steps: 94
Average Reward: -103.82
Success Rate: 57.00%
Best Reward: -4.00
length: 94
loss: -126.732955932

Episode 15440
Steps: 103
Average Reward: -94.55
Success Rate: 64.00%
Best Reward: -4.00
length: 103
loss: -150.60873413085938
--------------------------------------------------
Episode 15450
Steps: 148
Average Reward: -93.91
Success Rate: 64.00%
Best Reward: -4.00
length: 148
loss: 1.7224102020263672
--------------------------------------------------
Episode 15460
Steps: 228
Average Reward: -94.21
Success Rate: 65.00%
Best Reward: -4.00
length: 228
loss: -233.25161743164062
--------------------------------------------------
Episode 15470
Steps: 207
Average Reward: -94.67
Success Rate: 65.00%
Best Reward: -4.00
length: 207
loss: -77.144775390625
--------------------------------------------------
Episode 15480
Steps: 111
Average Reward: -91.96
Success Rate: 67.00%
Best Reward: -4.00
length: 111
loss: -35.71467208862305
--------------------------------------------------
Episode 15490
Steps: 139
Average Reward: -92.72
Success Rate: 66.00%
Best Reward: -4.00
length: 139
loss: -15.7699871063

Episode 15920
Steps: 91
Average Reward: -93.34
Success Rate: 65.00%
Best Reward: -4.00
length: 91
loss: -16.459333419799805
--------------------------------------------------
Episode 15930
Steps: 70
Average Reward: -90.03
Success Rate: 69.00%
Best Reward: -4.00
length: 70
loss: -32.04259490966797
--------------------------------------------------
Episode 15940
Steps: 53
Average Reward: -88.96
Success Rate: 68.00%
Best Reward: -4.00
length: 53
loss: -57.84850311279297
--------------------------------------------------
Episode 15950
Steps: 96
Average Reward: -93.24
Success Rate: 66.00%
Best Reward: -4.00
length: 96
loss: -10.740100860595703
--------------------------------------------------
Episode 15960
Steps: 51
Average Reward: -91.12
Success Rate: 67.00%
Best Reward: -4.00
length: 51
loss: -20.66966438293457
--------------------------------------------------
Episode 15970
Steps: 73
Average Reward: -88.71
Success Rate: 69.00%
Best Reward: -4.00
length: 73
loss: -61.25257873535156
-----

Episode 16400
Steps: 84
Average Reward: -91.09
Success Rate: 64.00%
Best Reward: -4.00
length: 84
loss: -114.37166595458984
--------------------------------------------------
Episode 16410
Steps: 58
Average Reward: -92.21
Success Rate: 66.00%
Best Reward: -4.00
length: 58
loss: -66.45977783203125
--------------------------------------------------
Episode 16420
Steps: 109
Average Reward: -92.57
Success Rate: 66.00%
Best Reward: -4.00
length: 109
loss: -43.39011001586914
--------------------------------------------------
Episode 16430
Steps: 181
Average Reward: -94.66
Success Rate: 66.00%
Best Reward: -4.00
length: 181
loss: 46.54304504394531
--------------------------------------------------
Episode 16440
Steps: 215
Average Reward: -94.17
Success Rate: 66.00%
Best Reward: -4.00
length: 215
loss: -128.73768615722656
--------------------------------------------------
Episode 16450
Steps: 34
Average Reward: -90.35
Success Rate: 67.00%
Best Reward: -4.00
length: 34
loss: -67.39891052246094


In [52]:
def new_select_action(self, state):
    state = torch.FloatTensor(state)
    # 添加探索机制
    if np.random.random() < 0.2:
        action = np.random.randint(0, 4)
        probs = self.policy(state)
        return action, torch.log(probs[action]+1e-8)
        
    probs = self.policy(state)
    distribution = Categorical(probs)
    action = distribution.sample()
    
    # 保存log概率用于后续训练
    return action.item(), distribution.log_prob(action)

# 替换方法
import types
agent.select_action = types.MethodType(new_select_action, agent)

In [14]:
import pickle

In [17]:
torch.save(agent.policy.state_dict(),'policy.pt')

In [49]:
import time
def visualize_agent(env, agent, num_episodes=5):
    """
    渲染显示智能体的行动
    """
    env = gym.make('CliffWalking-v0', render_mode='human')  # 创建可视化环境
    
    for episode in range(num_episodes):
        state_tuple = env.reset()
        state = state_tuple[0] if isinstance(state_tuple, tuple) else state_tuple
        total_reward = 0
        steps = 0
        done = False
        
        print(f"\nEpisode {episode + 1}")
        
        while not done:
            env.render()  # 渲染当前状态
            
            # 将状态转换为one-hot编码
            state_onehot = np.zeros(48)
            state_onehot[state] = 1
            
            # 使用训练好的策略选择动作
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state_onehot)
                probs = agent.policy(state_tensor)
                action = probs.argmax().item()  # 使用最可能的动作
            
            # 执行动作
            step_result = env.step(action)
            if len(step_result) == 4:
                next_state, reward, done, _ = step_result
            else:
                next_state, reward, terminated, truncated, _ = step_result
                done = terminated or truncated
            
            total_reward += reward
            steps += 1
            state = next_state
            
            # 添加小延迟使动作更容易观察
            time.sleep(0.5)
        
        print(f"Episode finished after {steps} steps. Total reward: {total_reward}")
    
    env.close()

# 在主程序最后添加：
if __name__ == "__main__":    
    # 训练完成后显示智能体行动
    print("\nVisualizing trained agent behavior...")
    visualize_agent(env, agent)


Visualizing trained agent behavior...

Episode 1
Episode finished after 15 steps. Total reward: -15

Episode 2
Episode finished after 15 steps. Total reward: -15

Episode 3
Episode finished after 15 steps. Total reward: -15

Episode 4
Episode finished after 15 steps. Total reward: -15

Episode 5
Episode finished after 15 steps. Total reward: -15


In [50]:
env.close()